# Privacy-Preserving RAG: Clinical Assistant with Tool Calling

This notebook implements a clinical assistant powered by OpenAI's O3 model that can answer patient-specific questions using structured data while preserving privacy through local de-identification.

## System Overview
- **OpenAI O3 Agent**: Acts as orchestrator and QA engine
- **Tool Functions**: Retrieve patient data from various sources
- **Local De-identifier**: Removes PHI before sending to OpenAI
- **Privacy-Safe Responses**: Clinically informed answers without compromising privacy

In [19]:
import pandas as pd

# Global variable for storage
df = None

def load_data(filepath):
    global df
    df = pd.read_csv(filepath)
    
    # Rename 'table_name' to 'note_type' to keep function logic the same
    df.rename(columns={"table_name": "note_type"}, inplace=True)

    # Clean any stray whitespace
    df["note_type"] = df["note_type"].str.strip().str.lower()
    df["patient_id"] = df["patient_id"].astype(str).str.strip()

    # Confirm presence of required columns
    if "note_type" not in df.columns or "patient_id" not in df.columns:
        raise ValueError("CSV must contain 'note_type' and 'patient_id' columns.")
    
    print("Data loaded with", df.shape[0], "rows.")



In [27]:
load_data("synthea_notes_top42_patients.csv")

Data loaded with 45191 rows.


In [28]:


def generate_notes_for_type(patient_id, note_type):
    if df is None:
        raise ValueError("Data not loaded. Run `load_data()` first.")
    
    patient_notes = df[
        (df["patient_id"] == patient_id) &
        (df["note_type"] == note_type)
    ]

    if patient_notes.empty:
        return f"No {note_type} notes found for patient {patient_id}."
    
    combined = "\n\n".join(patient_notes["note_text"].tolist())
    return f"{note_type.capitalize()} Notes for patient {patient_id}:\n\n{combined}"

In [2]:
# Install essential packages
# Run this cell first to install core dependencies

!pip install openai python-dotenv pandas numpy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Import necessary libraries
import os
import json
from typing import Dict, List, Any, Optional, Union
from datetime import datetime, date
import pandas as pd
import numpy as np

# OpenAI imports
from openai import OpenAI

# Environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ Libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully!
📅 Current date: 2025-08-01 21:15:38


In [4]:
# OpenAI Configuration
class ClinicalAssistantConfig:
    """Configuration for the Clinical Assistant"""
    
    def __init__(self):
        # OpenAI API configuration
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            print("⚠️  Warning: OPENAI_API_KEY not found in environment variables")
            print("💡 Please set your OpenAI API key in a .env file or environment variable")
        
        # Model configuration - using GPT-4o-mini for now (O3 models may not be available yet)
        self.model = "gpt-4o-mini"  # Using GPT-4o-mini until O3 is available
        self.max_completion_tokens = 4000  # Use max_completion_tokens for newer models
        self.temperature = 0.1  # Low temperature for clinical accuracy
        
        # Tool calling configuration
        self.max_tool_calls = 5
        self.parallel_tool_calls = True

# Initialize configuration
config = ClinicalAssistantConfig()

# Initialize OpenAI client
if config.api_key:
    client = OpenAI(api_key=config.api_key)
    print("✅ OpenAI client initialized successfully!")
    print(f"🤖 Model: {config.model}")
else:
    client = None
    print("❌ OpenAI client not initialized - API key required")

✅ OpenAI client initialized successfully!
🤖 Model: gpt-4o-mini


In [20]:
# Tool Function Definitions for OpenAI Function Calling
# These will be used by the O3 model to understand available tools

TOOL_DEFINITIONS = [
    {
        "type": "function",
        "function": {
            "name": "get_patient_observations",
            "description": "Retrieve laboratory test results and clinical observations for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_conditions",
            "description": "Retrieve patient diagnosis information, medical conditions, and medical history",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_medications",
            "description": "Retrieve current and past medications for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_careplans",
            "description": "Retrieve care plans and treatment plans for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_procedures",
            "description": "Retrieve medical procedures and interventions performed on a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_imaging_studies",
            "description": "Retrieve imaging studies and radiology reports for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_immunizations",
            "description": "Retrieve vaccination history and immunization records for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_patient_allergies",
            "description": "Retrieve allergy information and adverse reactions for a patient",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Unique patient identifier"
                    }
                },
                "required": ["patient_id"]
            }
        }
    }
]

print("✅ Tool function definitions created!")
print(f"🔧 Available tools: {len(TOOL_DEFINITIONS)}")
for tool in TOOL_DEFINITIONS:
    print(f"   - {tool['function']['name']}: {tool['function']['description']}")

✅ Tool function definitions created!
🔧 Available tools: 8
   - get_patient_observations: Retrieve laboratory test results and clinical observations for a patient
   - get_patient_conditions: Retrieve patient diagnosis information, medical conditions, and medical history
   - get_patient_medications: Retrieve current and past medications for a patient
   - get_patient_careplans: Retrieve care plans and treatment plans for a patient
   - get_patient_procedures: Retrieve medical procedures and interventions performed on a patient
   - get_patient_imaging_studies: Retrieve imaging studies and radiology reports for a patient
   - get_patient_immunizations: Retrieve vaccination history and immunization records for a patient
   - get_patient_allergies: Retrieve allergy information and adverse reactions for a patient


In [ ]:
# gated model login with Hugging Face CLI
# Make sure you have the Hugging Face CLI installed and authenticated
!pip install huggingface_hub
!pip install llama-cpp-python

from huggingface_hub import login
login(os.getenv("HUGGING_FACE_API_KEY"))

from llama_cpp import Llama

# Download the model to 'models/gemma-3-4b-it-qat-q4_0.gguf' as per previous steps
llm = Llama.from_pretrained(
    repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
    filename="gemma-3-4b-it-q4_0.gguf",  # ✅ correct filename
    n_ctx=2048,
    n_threads=8
)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


/Volumes/Nirvana/masters/AI in healthcare/final_project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_load_from_file_impl: using device Metal (Apple M2) - 10922 MiB free
llama_model_loader: loaded meta data with 39 key-value pairs and 444 tensors from /Users/arunjoshi/.cache/huggingface/hub/models--google--gemma-3-4b-it-qat-q4_0-gguf/snapshots/15f73f5eee9c28f53afefef5723e29680c2fc78a/./gemma-3-4b-it-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                      gemma3.context_length u32              = 131072
llama_model_loader: - kv   2:                         gemma

In [21]:
deidentifier_prompt = """
You are a local language model responsible for enforcing HIPAA compliance by identifying and 
removing all Protected Health Information (PHI) from clinical text and structured data before 
it is passed to an external system. Your task is to remove all 18 identifiers defined under 
HIPAA's Safe Harbor rule while preserving the clinical meaning of the data.

## HIPAA-Defined Identifiers to Remove/Mask

You must remove or mask the following 18 HIPAA-defined identifiers:

1. Names
2. Geographic locations smaller than a state (e.g., street address, city, ZIP code)
3. All elements of dates (except year) related to the individual (e.g., birthdate, admission/discharge, death, etc.)
4. Telephone numbers
5. Fax numbers
6. Email addresses
7. Social Security numbers
8. Medical record numbers
9. Health plan beneficiary numbers
10. Account numbers
11. Certificate or license numbers
12. Vehicle identifiers and license plates
13. Device identifiers and serial numbers
14. Web URLs
15. IP addresses
16. Biometric identifiers (e.g., fingerprints, voiceprints)
17. Full-face photos and comparable images
18. Any other unique identifying number, characteristic, or code

## Redaction Rules

- Replace identifiers with placeholders such as [REDACTED_NAME], [REDACTED_DATE], [REDACTED_ID]
- Convert absolute dates to relative expressions (e.g., "Post-op Day 2", "Admission Day 1") if possible
- Do not alter or summarize medical facts, terminology, or structure
- Do not generate synthetic replacements

## Compliance Requirements

- Do not preserve, paraphrase, or leak any PHI
- Do not miss or skip any of the 18 identifier categories
- Output must be fully HIPAA-compliant and safe for use with external LLMs
- If unsure about a possible identifier, err on the side of redacting it

## Goal

Produce a fully de-identified, privacy-safe version of the input that retains its clinical 
utility while strictly complying with all HIPAA Safe Harbor requirements.
"""

def deidentify(user_data):
    print(user_data);
    result = llm.create_chat_completion(
        messages=[
            {"role": "system", "content": [{"type": "text", "text": deidentifier_prompt}]},
            {"role": "user", "content": [{"type": "text", "text": user_data}]}
        ],
        max_tokens=1000,
        temperature=0.3
    )
    print("Deidentification complete.")
    print("Deidentified content:", result["choices"][0]["message"]["content"])
    return result["choices"][0]["message"]["content"]

In [22]:
# Tool Function Implementations (Using generate_notes_for_type)
class ClinicalDataTools:
    """Clinical data retrieval tools for the OpenAI assistant"""
    
    def __init__(self):
        self.de_identifier = None  # Will be initialized later
        print("🔧 Clinical Data Tools initialized with all available note types")
    
    def get_patient_observations(self, patient_id: str) -> str:
        """
        Retrieve laboratory test results and clinical observations for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "observations")
    
    def get_patient_conditions(self, patient_id: str) -> str:
        """
        Retrieve patient diagnosis information, medical conditions, and medical history
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "conditions")
    
    def get_patient_medications(self, patient_id: str) -> str:
        """
        Retrieve current and past medications for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "medications")
    
    def get_patient_careplans(self, patient_id: str) -> str:
        """
        Retrieve care plans and treatment plans for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "careplans")
    
    def get_patient_procedures(self, patient_id: str) -> str:
        """
        Retrieve medical procedures and interventions performed on a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "procedures")
    
    def get_patient_imaging_studies(self, patient_id: str) -> str:
        """
        Retrieve imaging studies and radiology reports for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "imaging_studies")
    
    def get_patient_immunizations(self, patient_id: str) -> str:
        """
        Retrieve vaccination history and immunization records for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "immunizations")
    
    def get_patient_allergies(self, patient_id: str) -> str:
        """
        Retrieve allergy information and adverse reactions for a patient
        
        Args:
            patient_id: Unique patient identifier
        """
        return generate_notes_for_type(patient_id, "allergies")

# Initialize the tools
clinical_tools = ClinicalDataTools()
print("✅ Clinical tools instance created with all available note types!")
print("📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies")

🔧 Clinical Data Tools initialized with all available note types
✅ Clinical tools instance created with all available note types!
📋 Available note types: observations, conditions, medications, careplans, procedures, imaging_studies, immunizations, allergies


In [23]:
# LLM Call and Prompt Management Functions
def build_system_prompt() -> str:
    """Build the system prompt for the clinical assistant"""
    return f"""You are an AI Clinical Reasoning Assistant with expertise in internal medicine. 
    Analyze clinical data and respond to patient-specific questions using structured reasoning.

    Available Tools:
    - get_patient_observations(): Laboratory test results and clinical observations
    - get_patient_conditions(): Diagnosis history, medical conditions, and medical history
    - get_patient_medications(): Current and past medications
    - get_patient_careplans(): Care plans and treatment plans
    - get_patient_procedures(): Medical procedures and interventions
    - get_patient_imaging_studies(): Imaging studies and radiology reports
    - get_patient_immunizations(): Vaccination history and immunization records
    - get_patient_allergies(): Allergy information and adverse reactions

    Clinical Reasoning Framework:
    1. Analyze the question to determine needed information
    2. Use appropriate tools to gather relevant patient data
    3. Synthesize findings from multiple data sources
    4. Provide clear, evidence-based responses with clinical reasoning

    Rules:
    - Use ONLY data provided by tool outputs
    - Reference relative timeframes when provided (e.g., "Day 0", "Post-op Day 3")
    - Acknowledge limitations if data is insufficient
    - Suggest additional information needed when applicable
    - Consider interactions between medications, conditions, and procedures
    - Always check for allergies before recommending treatments

    Current date: {datetime.now().strftime('%Y-%m-%d')}
    """

def build_messages(query: str, patient_id: str = None) -> List[Dict[str, str]]:
    """Build messages for the LLM call"""
    messages = [
        {"role": "system", "content": build_system_prompt()},
        {"role": "user", "content": f"Clinical query: {query}"}
    ]
    
    if patient_id:
        messages[-1]["content"] += f"\\nPatient ID: {patient_id}"
    
    return messages

def call_llm(client: OpenAI, config: ClinicalAssistantConfig, messages: List[Dict[str, str]]):
    """Make the actual LLM call"""
    return client.chat.completions.create(
        model=config.model,
        messages=messages,
        tools=TOOL_DEFINITIONS,
        tool_choice="auto",
        max_completion_tokens=config.max_completion_tokens,
        temperature=config.temperature
    )

In [24]:
# Clinical Assistant with Optional De-identification
class ClinicalAssistant:
    """Main clinical assistant that orchestrates OpenAI GPT-4o-mini model with tool calling"""
    
    def __init__(self, client: OpenAI, config: ClinicalAssistantConfig, tools: ClinicalDataTools):
        self.client = client
        self.config = config
        self.tools = tools
        
        # Map function names to actual methods
        self.function_map = {
            "get_patient_observations": self.tools.get_patient_observations,
            "get_patient_conditions": self.tools.get_patient_conditions,
            "get_patient_medications": self.tools.get_patient_medications,
            "get_patient_careplans": self.tools.get_patient_careplans,
            "get_patient_procedures": self.tools.get_patient_procedures,
            "get_patient_imaging_studies": self.tools.get_patient_imaging_studies,
            "get_patient_immunizations": self.tools.get_patient_immunizations,
            "get_patient_allergies": self.tools.get_patient_allergies
        }
    
    def de_identify_data(self, data: str) -> str:
        """De-identify text data using the existing deidentify function"""
        try:
            return deidentify(data)
        except Exception as e:
            print(f"Warning: De-identification failed: {e}")
            return data
    
    def execute_tool_call(self, function_name: str, arguments: Dict[str, Any], apply_deidentification: bool = False) -> str:
        """Execute a tool function call with optional de-identification"""
        if function_name not in self.function_map:
            return f"Error: Unknown function: {function_name}"
        print(f"Executing tool call: {function_name}")
        try:
            func = self.function_map[function_name]
            raw_result = func(**arguments)
            
            # Apply de-identification if requested
            if apply_deidentification and isinstance(raw_result, str):
                return self.de_identify_data(raw_result)
            
            return raw_result
            
        except Exception as e:
            return f"Error executing {function_name}: {str(e)}"
    
    def process_clinical_query(self, query: str, patient_id: str = None, apply_deidentification: bool = False) -> str:
        """
        Process a clinical query using OpenAI GPT-4o-mini with tool calling
        
        Args:
            query: The clinical question to answer
            patient_id: Optional patient ID if known
            apply_deidentification: Whether to apply de-identification to tool outputs
            
        Returns:
            Clinical assistant response
        """
        if not self.client:
            return "❌ OpenAI client not initialized. Please check your API key."
        
        try:
            # Get initial response from LLM
            messages = build_messages(query, patient_id)
            response = call_llm(self.client, self.config, messages)
            
            # Handle tool calls if any
            if response.choices[0].message.tool_calls:
                current_messages = messages.copy()
                current_messages.append({
                    "role": "assistant",
                    "content": response.choices[0].message.content,
                    "tool_calls": response.choices[0].message.tool_calls
                })
                
                # Process each tool call
                for tool_call in response.choices[0].message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    # Execute tool call with optional de-identification
                    tool_result = self.execute_tool_call(function_name, function_args, apply_deidentification)
                    
                    current_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": tool_result
                    })
                
                # Get final response with tool results
                final_response = call_llm(self.client, self.config, current_messages)
                return final_response.choices[0].message.content
            else:
                return response.choices[0].message.content
            
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the clinical assistant
if client:
    assistant = ClinicalAssistant(client, config, clinical_tools)
    print("✅ Clinical Assistant initialized with all 8 tool functions!")
else:
    assistant = None

✅ Clinical Assistant initialized with all 8 tool functions!


In [25]:
# Test Clinical Assistant with Integrated De-identification
print("🧪 Testing Clinical Assistant with Integrated De-identification")
print("=" * 60)

# Sample clinical queries to test the system with all available tools
sample_queries = [
    "What are this patient's current lab abnormalities and what do they suggest clinically?",
    "Can you summarize this patient's medical conditions and current medications?",
    "What procedures has this patient undergone and what was the care plan?",
    "Does this patient have any allergies I should be aware of before prescribing?",
    "What is this patient's vaccination status and immunization history?",
    "What imaging studies have been performed and what were the findings?",
    "Based on all available data, what is the comprehensive clinical picture?",
    "Are there any drug interactions or contraindications based on current medications and allergies?"
]

# Enhanced interactive testing function with de-identification options
def test_query(query_text: str, patient_id: str = "12345678", apply_deidentification: bool = True):
    """
    Test function for interactive querying with de-identification options
    
    Args:
        query_text: The clinical question to ask
        patient_id: Patient identifier (default: "12345678")
        apply_deidentification: Whether to apply de-identification (default: True)
    
    Returns:
        Assistant response
    """
    if not assistant:
        return "❌ Clinical Assistant not initialized. Please set your OpenAI API key."
    
    print(f"🔍 Patient ID: {patient_id}")
    print(f"📝 Query: {query_text}")
    print(f"🔒 De-identification: {'Enabled' if apply_deidentification else 'Disabled'}")
    print("\n🤖 Assistant Response:")
    print("-" * 60)
    
    response = assistant.process_clinical_query(query_text, patient_id, apply_deidentification)
    print(response)
    return response

🧪 Testing Clinical Assistant with Integrated De-identification


In [29]:


# Test with a sample patient ID (this will be de-identified)
test_patient_id = "e7a5d3dc-3484-a24e-6ef3-0737b403f950"
query = sample_queries[6]
test_query(query, test_patient_id, apply_deidentification=True)

🔍 Patient ID: e7a5d3dc-3484-a24e-6ef3-0737b403f950
📝 Query: Based on all available data, what is the comprehensive clinical picture?
🔒 De-identification: Enabled

🤖 Assistant Response:
------------------------------------------------------------
Executing tool call: get_patient_observations
Observations Notes for patient e7a5d3dc-3484-a24e-6ef3-0737b403f950:

patient_id: e7a5d3dc-3484-a24e-6ef3-0737b403f950; full_name: Marni Tremblay; observation_date: 2016-10-26 09:00:18; birthdate: 1976-10-20; ssn: 999-36-6337; drivers: S99999779; passport: X42697334X; prefix: Mrs.; marital: D; race: white; ethnicity: nonhispanic; gender: F; birthplace: Park River  North Dakota  US; patient_address: 118 Kiehn Gardens Suite 40; patient_city: West Fargo; patient_state: North Dakota; county: Cass County; fips: 38017; patient_zip: 58078; lat: 46.893402133270634; lon: -96.86213906026983; healthcare_expenses: 157011.5; healthcare_coverage: 1055670.27; income: 29501; age_at_observation: 40; observation_cate

Llama.generate: 1063 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   13279.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   39812.65 ms /   483 runs   (   82.43 ms per token,    12.13 tokens per second)
llama_perf_context_print:       total time =   40165.96 ms /   484 tokens


Deidentification complete.
Deidentified content: patient_id: [REDACTED_ID]; full_name: [REDACTED_NAME]; imaging_study_date: Post-op Day 2; birthdate: 1976-10-20; ssn: [REDACTED_SSN]; drivers: [REDACTED_LICENSE]; passport: [REDACTED_PASSPORT]; prefix: Mrs.; marital: D; race: white; ethnicity: nonhispanic; gender: F; birthplace: Park River North Dakota US; patient_address: 118 Kiehn Gardens Suite 40; patient_city: [REDACTED_CITY]; patient_state: [REDACTED_STATE]; county: Cass County; fips: 38017; patient_zip: [REDACTED_ZIP]; lat: [REDACTED_LAT]; lon: [REDACTED_LON]; healthcare_expenses: 157011.5; healthcare_coverage: 1055670.27; income: 29501; age_at_imaging: 43; bodysite_code: 51185008; bodysite_description: Thoracic structure (body structure); modality_code: DX; modality_description: Digital Radiography; sop_code: 1.2.840.10008.5.1.4.1.1.1.1; sop_description: Digital X-Ray Image Storage; procedure_code: 399208008; encounter_class: emergency; encounter_description: Emergency room admiss

'❌ Error processing query: Error code: 400 - {\'error\': {\'message\': "This model\'s maximum context length is 128000 tokens. However, your messages resulted in 179578 tokens (179300 in the messages, 278 in the functions). Please reduce the length of the messages or functions.", \'type\': \'invalid_request_error\', \'param\': \'messages\', \'code\': \'context_length_exceeded\'}}'